# PlayHQ Fixture Scraping

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ssardina/tapp-fixture/blob/main/playhq_scrape.ipynb)

This system allows to scrape game fixtures from [PlayHQ](http://playhq.com/) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs). It will produce a CSV file ready to be uploaded as Schedule in [TeamApp](https://brunswickmagicbasketball.teamapp.com/).

The *Public* APIs only require a header parameters to get a successful response, which includes the following components:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com. This key can be stored in a file `x_api_key.txt` or it will be asked interactively by the notebook otherwise. In many cases, the feature to create new API credentials is disabled for a user and can only be actioned by a Super Administrator role within the Play HQ portal.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.


Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

**Contact:** Sebastian Sardina (sssardina@gmail.com)

In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import re
import os
import calendar, datetime
import dtale

import utils
import playhq as phq

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## 1. Configuration and set-up

We first configure and set-up the application. This means reading configuration variables from a config file and setting the game day.

So, first of all, specify the following information:

1. Configuration file for the club and season.
2. Game dates interval to scrape.

In [2]:
# Change this to import your club's own configuration
# from config_bmc_w23 import *
# from config_bmc_s23 import *
from config_cba_24 import *

# Set the game date interval scrape
GAME_DATE_START = datetime.date.today() # by default, any game after today
# GAME_DATE_START = utils.next_day(calendar.SATURDAY)   # start from next game day (used for Domestic)
# GAME_DATE_START = datetime.date(2024, 3, 25) # start on a specific day - (used for Rep - REP ROUND 5+)

WEEKS = 10   # how many weeks after date start we want to scrape (use 1 for just next game)
GAME_DATE_END = GAME_DATE_START + datetime.timedelta(days=WEEKS*7)


###############################################################
# DO NOT CHANGE FROM HERE
###############################################################

# Get nice game date format: Saturday August 06, 2022
GAME_DATE_START_TIMESTAMP = pd.to_datetime(GAME_DATE_START).tz_localize(TIMEZONE)
GAME_DATE_END_TIMESTAMP = pd.to_datetime(GAME_DATE_END).tz_localize(TIMEZONE)

GAME_DATE_START_NAME = utils.pretty_date(GAME_DATE_START_TIMESTAMP)
GAME_DATE_END_NAME = utils.pretty_date(GAME_DATE_END_TIMESTAMP)

# Create phq_club object
phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE, tapp_team_name, tapp_game_name)
if SEASON_ID is None:
    raise SystemExit("ERROR! Please specify either SEASON_ID.")
SEASON_NAME = phq_club.get_season_name(SEASON_ID)

print(f"Club name: {CLUB_NAME} (org. id: {ORG_ID})")
print(f"Season: {SEASON_NAME} (season id: {SEASON_ID})")
print("X-tenant:", X_TENANT, "x-api-key:", X_API_KEY)
print("Output path:", OUTPUT_PATH)
if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

print("Timezone:", TIMEZONE)
print(f"Game dates: {GAME_DATE_START_NAME} - {GAME_DATE_END_NAME}")
print("PlayHQ Club fixture:", PLAYHQ_SEASON_URL)

Club name: Coburg Giants Basketball Club (org. id: 5db1d983-5453-4b73-912a-457e72c273c3)
Season: Victorian Junior Basketball League (season id: e0b8b997-f2fc-43fc-88de-d53bc209ebd9)
X-tenant: bv x-api-key: f5d33c76-f858-49fa-8330-8e0e396219cd
Output path: Coburg_Rep/2023-24/
Timezone: Australia/Melbourne
Game dates: Monday March 25, 2024 (2024/03/25) - Monday June 03, 2024 (2024/06/03)
PlayHQ Club fixture: https://bit.ly/cba-vjbl24


## 2. Get upcoming games for club's teams

First, get the teams of the club, sort them based on age group.

In [3]:
teams_df = phq_club.get_season_teams(SEASON_ID)
teams_df.sort_values('age', ascending=True, inplace=True)
teams_df.reset_index(inplace=True, drop=True)

teams = teams_df['name'].values
print(f"Found {len(teams)} teams:", teams)

# teams_df

Found 35 teams: ['Coburg U12 Boys 1' 'Coburg U12 Boys 2' 'Coburg U12 Boys 3'
 'Coburg U12 Boys 4' 'Coburg U12 Boys 5' 'Coburg U12 Girls 1'
 'Coburg U12 Girls 2' 'Coburg U12 Girls 3' 'Coburg U14 Girls 4'
 'Coburg U14 Girls 3' 'Coburg U14 Girls 2' 'Coburg U14 Girls 1'
 'Coburg U14 Girls 5' 'Coburg U14 Boys 4' 'Coburg U14 Boys 3'
 'Coburg U14 Boys 2' 'Coburg U14 Boys 1' 'Coburg U14 Boys 5'
 'Coburg U16 Girls 4' 'Coburg U16 Girls 3' 'Coburg U16 Girls 2'
 'Coburg U16 Girls 1' 'Coburg U16 Boys 6' 'Coburg U16 Boys 4'
 'Coburg U16 Boys 3' 'Coburg U16 Boys 2' 'Coburg U16 Boys 1'
 'Coburg U16 Boys 5' 'Coburg U18 Girls 2' 'Coburg U18 Boys 1'
 'Coburg U18 Boys 2' 'Coburg U18 Boys 3' 'Coburg U18 Boys 4'
 'Coburg U18 Girls 1' 'Coburg U20 Boys 1']


In [12]:
# teams_df.query("name == 'Coburg U12 Boys 1'")

# if needed keep only a specific team (e.g., for debugging later)
# teams_df = teams_df.query("id == '115ca2a9-5383-4beb-9703-691006568974'")

teams_df = teams_df.query("name == 'Coburg U14 Girls 4' or name == 'Coburg U14 Girls 5'")
teams_df

,id,name,grade.id,grade.name,grade.url,age
8,6f8b352a-6a8e-476b-849c-7b8f3caa6979,Coburg U14 Girls 4,9a9591e5-5148-422d-9dba-81d4bb374667,U14 Girls - VJL9 (P1),https://www.playhq.com/basketball-victoria/org...,14
12,df9dd513-7cc0-41d5-ac43-978717f5834d,Coburg U14 Girls 5,16cd00dc-4a9d-44b2-a969-1f5e939e72da,U14 Girls - VJL10 (P1),https://www.playhq.com/basketball-victoria/org...,14


Next, extract all games between the dates specified for these teams of the club.

In [13]:
upcoming_games_df = phq_club.get_games(teams_df, GAME_DATE_START_TIMESTAMP, GAME_DATE_END_TIMESTAMP)

if upcoming_games_df is not None:
    print(f'There were {upcoming_games_df.shape[0]} games extracted for game between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')
    upcoming_games_df[phq.GAMES_COLS]
else:
    print(f'No games between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')

2024-04-07 06:33:51 INFO Games extracted for team: Coburg U14 Girls 4
2024-04-07 06:33:51 INFO Games extracted for team: Coburg U14 Girls 5


There were 11 games extracted for game between Monday March 25, 2024 (2024/03/25) and Monday June 03, 2024 (2024/06/03)


If in FINALS, there may be games scheduled for the following weekend, so they have no opponent yet.
We now list them to check and then drop them as they are not yet actual games.

In [14]:
# get the teams that have missing competitor and remove them from upcoming games
mask_no_competitors = upcoming_games_df['competitors'].apply(lambda x: len(x) == 1)

teams_pending_competitors = upcoming_games_df[mask_no_competitors].team_name.values
upcoming_games_df.drop(upcoming_games_df[mask_no_competitors].index, inplace=True)

print("Teams that have a pending competitor (finals?):")
teams_pending_competitors

Teams that have a pending competitor (finals?):


array([], dtype=object)

Show final upcoming games before converting to TeamApps.

In [15]:
# dtale.show(upcoming_games_df)
print("No of upcoming games:", upcoming_games_df.shape[0])

upcoming_games_df

No of upcoming games: 11


,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceName,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp
0,2b3b6ae9-f4aa-4e89-aeb7-1b93c833cb50,Coburg U14 Girls 4,6f8b352a-6a8e-476b-849c-7b8f3caa6979,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-14 14:20:47+11:00,None,[{'id': '72841132-993c-4c2c-9dd4-85d161a9062d'...,9a9591e5-5148-422d-9dba-81d4bb374667,...,Court 3,Crt3,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2024-04-19 19:40:00+10:00
1,9c2b0e07-74ea-43db-bcd0-3b140bbca254,Coburg U14 Girls 4,6f8b352a-6a8e-476b-849c-7b8f3caa6979,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-04-02 14:58:01+11:00,None,[{'id': '6f8b352a-6a8e-476b-849c-7b8f3caa6979'...,9a9591e5-5148-422d-9dba-81d4bb374667,...,Court 1,Crt1,18 Cemetery Road,3033,Keilor East,VIC,Australia,-37.728570,144.851160,2024-04-26 18:40:00+10:00
2,3f6a539b-acfc-45aa-8bf2-ba7f8327af2f,Coburg U14 Girls 4,6f8b352a-6a8e-476b-849c-7b8f3caa6979,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-20 11:34:11+11:00,None,[{'id': '1f8161b7-3e26-49a4-80d6-be13e8f6ffa7'...,9a9591e5-5148-422d-9dba-81d4bb374667,...,Court 3,Crt3,39 Lily Street,3019,Braybrook,VIC,Australia,-37.783337,144.848348,2024-05-03 20:40:00+10:00
3,8e3ec687-32e6-470b-b70d-4683fb7c81f6,Coburg U14 Girls 4,6f8b352a-6a8e-476b-849c-7b8f3caa6979,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-22 11:37:01+11:00,None,[{'id': 'abcef655-fb8d-4756-88b0-8535247fc5be'...,9a9591e5-5148-422d-9dba-81d4bb374667,...,Court 1,Crt1,44 Civic Drive,3088,Greensborough,VIC,Australia,-37.689450,145.111540,2024-05-17 19:40:00+10:00
4,d5c386c4-c40e-4004-912a-393a7f12407d,Coburg U14 Girls 4,6f8b352a-6a8e-476b-849c-7b8f3caa6979,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-25 12:23:09+11:00,None,[{'id': '6f8b352a-6a8e-476b-849c-7b8f3caa6979'...,9a9591e5-5148-422d-9dba-81d4bb374667,...,Court 4,Crt4,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2024-05-24 18:40:00+10:00
5,03866968-560b-476a-bbf4-2b9d981301a5,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-15 10:56:48+11:00,None,[{'id': 'df9dd513-7cc0-41d5-ac43-978717f5834d'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 3,Crt3,9 Humphreys Way,3178,Rowville,VIC,Australia,-37.920179,145.265652,2024-04-19 18:40:00+10:00
6,eb234449-f07e-4bf2-92b6-d1970ca7122c,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-20 14:00:45+11:00,None,[{'id': 'df9dd513-7cc0-41d5-ac43-978717f5834d'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 4,Crt4,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2024-04-26 19:40:00+10:00
7,cd8f781c-77e9-4f57-b6f9-3bb393bbab8d,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-20 13:53:16+11:00,None,[{'id': '442c5c9b-436f-4fb6-a5b4-73c1deee6832'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 13,Crt13,291 George Street,3152,Wantirna South,VIC,Australia,-37.880650,145.211680,2024-05-03 19:40:00+10:00
8,af13728d-70ff-4b5b-91cb-af67aa547aad,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-21 13:20:29+11:00,None,[{'id': '38876544-66bb-44f5-a3f8-dba4df2ba7df'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 4,Crt4,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2024-05-10 20:40:00+10:00
9,54

In [17]:
# upcoming_games_df.query("team_name == 'Magic U14 Boys Diamond'")
# upcoming_games_df.loc[upcoming_games_df.team_name == 'Magic U12 Girls White']
# upcoming_games_df.iloc[0,:]

upcoming_games_df.query("team_name == 'Coburg U14 Girls 5'")

,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceName,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp
5,03866968-560b-476a-bbf4-2b9d981301a5,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-15 10:56:48+11:00,None,[{'id': 'df9dd513-7cc0-41d5-ac43-978717f5834d'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 3,Crt3,9 Humphreys Way,3178,Rowville,VIC,Australia,-37.920179,145.265652,2024-04-19 18:40:00+10:00
6,eb234449-f07e-4bf2-92b6-d1970ca7122c,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-20 14:00:45+11:00,None,[{'id': 'df9dd513-7cc0-41d5-ac43-978717f5834d'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 4,Crt4,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2024-04-26 19:40:00+10:00
7,cd8f781c-77e9-4f57-b6f9-3bb393bbab8d,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-20 13:53:16+11:00,None,[{'id': '442c5c9b-436f-4fb6-a5b4-73c1deee6832'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 13,Crt13,291 George Street,3152,Wantirna South,VIC,Australia,-37.880650,145.211680,2024-05-03 19:40:00+10:00
8,af13728d-70ff-4b5b-91cb-af67aa547aad,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-21 13:20:29+11:00,None,[{'id': '38876544-66bb-44f5-a3f8-dba4df2ba7df'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 4,Crt4,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2024-05-10 20:40:00+10:00
9,54905b2b-394d-45fb-9d17-2d3654108d13,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-22 14:22:11+11:00,None,[{'id': '79c56704-0243-4ffd-8424-3f407f394186'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 2,Crt2,11 Grosvenor Street,3130,Blackburn North,VIC,Australia,-37.802924,145.162045,2024-05-17 19:40:00+10:00
10,54e2ca39-b803-4830-a70b-eb7eb5d0a0cb,Coburg U14 Girls 5,df9dd513-7cc0-41d5-ac43-978717f5834d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2024-03-04 22:54:13+11:00,2024-03-25 12:23:04+11:00,None,[{'id': '0a84762d-122a-44c8-898d-313894f5769b'...,16cd00dc-4a9d-44b2-a969-1f5e939e72da,...,Court 1,Crt1,857 Plenty Road,3073,Reservoir,VIC,Australia,-37.721430,145.030090,2024-05-24 19:40:00+10:00


## 3. Convert to TeamApp CSV format

Next, we convert the PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

This process takes time as it processes game per game and even obtains short URL links for each game.

In [18]:
if upcoming_games_df is None:
    raise SystemExit("There are no games to process. Exiting.")

games_tapps_df = phq_club.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP, game_duration=45)
print("Done computing the games for Teams App")

# find out the game day if there is one day all teams play on only
game_day = None
single_game_day = (games_tapps_df['start_date'].drop_duplicates().size == 1)
if single_game_day:
    game_day = games_tapps_df.iloc[0]['start_date']
    print("All games are in the following day:", utils.pretty_date(game_day))

Done computing the games for Teams App


Inspect how the description of one of the games will look like:

In [19]:
# games_tapps_df.sample(3)
# dtale.show(games_tapps_df)
games_tapps_df

,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing,opponent,court
0,Game 14.4 Girls - Round 4,14.4 Girls,2024-04-19,2024-04-19,19:40:00,20:25:00,RSVP mandatory for the game.\n\nOpponent: Bull...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",14.4 Girls,1,1,0,1,0,Bulleen U14 Girls 5,Court 3
1,Game 14.4 Girls - Round 5,14.4 Girls,2024-04-26,2024-04-26,18:40:00,19:25:00,RSVP mandatory for the game.\n\nOpponent: Keil...,Brimbank Anglican Church,"18 Cemetery Road, Keilor East",14.4 Girls,1,1,0,1,0,Keilor U14 Girls 7,Court 1
2,Game 14.4 Girls - Round 6,14.4 Girls,2024-05-03,2024-05-03,20:40:00,21:25:00,RSVP mandatory for the game.\n\nOpponent: West...,RecWest Braybrook,"39 Lily Street, Braybrook",14.4 Girls,1,1,0,1,0,Westgate U14 Girls 3,Court 3
3,Game 14.4 Girls - Round 8,14.4 Girls,2024-05-17,2024-05-17,19:40:00,20:25:00,RSVP mandatory for the game.\n\nOpponent: Diam...,Diamond Valley Sports and Fitness Centre,"44 Civic Drive, Greensborough",14.4 Girls,1,1,0,1,0,Diamond Valley U14 Girls 8,Court 1
4,Game 14.4 Girls - Round 9,14.4 Girls,2024-05-24,2024-05-24,18:40:00,19:25:00,RSVP mandatory for the game.\n\nOpponent: Sunb...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",14.4 Girls,1,1,0,1,0,Sunbury U14 Girls 3,Court 4
5,Game 14.5 Girls - Round 4,14.5 Girls,2024-04-19,2024-04-19,18:40:00,19:25:00,RSVP mandatory for the game.\n\nOpponent: Keys...,Rowville Secondary College - East Campus,"9 Humphreys Way, Rowville",14.5 Girls,1,1,0,1,0,Keysborough U14 Girls 2,Court 3
6,Game 14.5 Girls - Round 5,14.5 Girls,2024-04-26,2024-04-26,19:40:00,20:25:00,RSVP mandatory for the game.\n\nOpponent: Camb...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",14.5 Girls,1,1,0,1,0,Camberwell U14 Girls 6,Court 4
7,Game 14.5 Girls - Round 6,14.5 Girls,2024-05-03,2024-05-03,19:40:00,20:25:00,RSVP mandatory for the game.\n\nOpponent: Knox...,State Basketball Centre,"291 George Street, Wantirna South",14.5 Girls,1,1,0,1,0,Knox U14 Girls 8,Court 13
8,Game 14.5 Girls - Round 7,14.5 Girls,2024-05-10,2024-05-10,20:40:00,21:25:00,RSVP mandatory for the game.\n\nOpponent: Elth...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",14.5 Girls,1,1,0,1,0,Eltham U14 Girls 7,Court 4
9,Game 14.5 Girls - Round 8,14.5 Girls,2024-05-17,2024-05-17,19:40:00,20:25:00,RSVP mandatory for the game.\n\nOpponent: Blac...,Slater Reserve Stadium,"11 Grosvenor Street, Blackburn North",14.5 Girls,1,1,0,1,0,Blackburn U14 Girls 3,Court 2


Keep games after a particular date:

In [ ]:
import datetime

START_DATE = None
# START_DATE = datetime.date(2023, 7, 21)

if START_DATE is not None:
    print("Keeping games after:", START_DATE)
    games_tapps_df[games_tapps_df['start_date'] > START_DATE]
    games_tapps_df = games_tapps_df[games_tapps_df['start_date'] > START_DATE]
else:
    print("Keeping all games")
games_tapps_df

In [ ]:
# Inspect description game of one team
# team = "12.2 Boys"
team = "U14 Boys Black"

print("Description for:", team)
print(games_tapps_df.query("team_name == @team")['description'].values[0])

## 4. Append BYE games (if necessary)

We generate BYE entries for TeamsAPP ***only*** if the games are played all in the same day.

In [24]:
days_games = games_tapps_df['start_date'].drop_duplicates().values

if len(days_games) == 1:
    game_day = days_games[0]
    print("Seems all games are played on:", game_day)


In [25]:
bye_teams= False    # assume no bye games

if game_day is not None:
    game_day = games_tapps_df.iloc[0]['start_date']

    # Extract the date of the round
    # date = games_tapps_df.iloc[1]['start_date']
    print(f"Extract BYE games for games on {utils.pretty_date(game_day)}")

    playing_teams = upcoming_games_df['team_id'].tolist()
    bye_teams = teams_df.loc[~teams_df['id'].isin(playing_teams)]['name'].tolist()
    bye_teams = list(map(lambda x: tapp_team_name(x), bye_teams))

    if bye_teams:
        games_bye_df = phq_club.build_teamsapp_bye_schedule(bye_teams, game_day, DESC_BYE_TAPP)
        print(f"Bye teams ({len(bye_teams)}): ", bye_teams)

        games_tapps_df = pd.concat([games_tapps_df, games_bye_df])
        games_tapps_df.drop_duplicates(inplace=True)
        games_tapps_df.reset_index(inplace=True, drop=True)
    else:
        print("No BYE games this round...")
else:
    print("Games obtained are not on the same day, not computing BYE games...")

(bye_teams and games_bye_df)

Games obtained are not on the same day, not computing BYE games...


False

## 5. Final review

Finally, we report the games to be written into Schedule CSV file and **CHECKING THAT ALL IS GOOD TO GO!**

Particularly, look for games that are schedule but **PENDING** and without all details (time or venue).

In [20]:
games_tapps_df.columns
games_tapps_df[['event_name', 'team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
# games_tapps_df

,event_name,team_name,opponent,start_date,start_time,venue,court
0,Game 14.4 Girls - Round 4,14.4 Girls,Bulleen U14 Girls 5,2024-04-19,19:40:00,Coburg Basketball Stadium,Court 3
1,Game 14.4 Girls - Round 5,14.4 Girls,Keilor U14 Girls 7,2024-04-26,18:40:00,Brimbank Anglican Church,Court 1
2,Game 14.4 Girls - Round 6,14.4 Girls,Westgate U14 Girls 3,2024-05-03,20:40:00,RecWest Braybrook,Court 3
3,Game 14.4 Girls - Round 8,14.4 Girls,Diamond Valley U14 Girls 8,2024-05-17,19:40:00,Diamond Valley Sports and Fitness Centre,Court 1
4,Game 14.4 Girls - Round 9,14.4 Girls,Sunbury U14 Girls 3,2024-05-24,18:40:00,Coburg Basketball Stadium,Court 4
5,Game 14.5 Girls - Round 4,14.5 Girls,Keysborough U14 Girls 2,2024-04-19,18:40:00,Rowville Secondary College - East Campus,Court 3
6,Game 14.5 Girls - Round 5,14.5 Girls,Camberwell U14 Girls 6,2024-04-26,19:40:00,Coburg Basketball Stadium,Court 4
7,Game 14.5 Girls - Round 6,14.5 Girls,Knox U14 Girls 8,2024-05-03,19:40:00,State Basketball Centre,Court 13
8,Game 14.5 Girls - Round 7,14.5 Girls,Eltham U14 Girls 7,2024-05-10,20:40:00,Coburg Basketball Stadium,Court 4
9,Game 14.5 Girls - Round 8,14.5 Girls,Blackburn U14 Girls 3,2024-05-17,19:40:00,Slater Reserve Stadium,Court 2


We stop the execution here if we are running all Jupyter notebook.

In [ ]:
raise SystemExit("Stop right there! Continue below to produce the CSV file if needed.")

## 6. Save to CSV file for Teams App import

OK we are ready to import into Teams App.

### 6.2. Check changes with previous saves

If the schedule was generated before, check if the new one differs with the one saved already.

First, let us define the files that we will save to disk.

In [21]:
now = datetime.datetime.now() # current date and time
now_str = now.strftime("%Y_%m_%d-%H:%M:%S")

id_file = now_str
if game_day is not None:    # there is one date for all games!
    id_file = utils.compact_date(game_day)

file_csv = os.path.join(OUTPUT_PATH, f"schedule-teamsapp-{id_file}.csv")
file_upcoming_pkl = os.path.join(OUTPUT_PATH, f"upcoming_games_df-{id_file}.pkl")
file_games_tapps = os.path.join(OUTPUT_PATH, f"games_tapps_df-{id_file}.pkl")

print("Files to save:")
print(file_csv)
print(file_upcoming_pkl)
print(file_games_tapps)

if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

Files to save:
Coburg_Rep/2023-24/schedule-teamsapp-2024_04_07-06:34:52.csv
Coburg_Rep/2023-24/upcoming_games_df-2024_04_07-06:34:52.pkl
Coburg_Rep/2023-24/games_tapps_df-2024_04_07-06:34:52.pkl


Next, let's check if there was a saved file for the upcoming game day.

In [22]:
cols = ['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']

changed_games_df = None
if os.path.exists(file_games_tapps):
    print("There was already a schedule saved, recovering it to compare...")
    old_games_tapps_df = pd.read_pickle(file_games_tapps)

    teams_changed = pd.concat([games_tapps_df[cols], old_games_tapps_df[cols]]).drop_duplicates(keep=False)['team_name'].unique()
    print("Teams whose games have changed (updated, new, dropped):", teams_changed)

    old_games_df = old_games_tapps_df[cols].query("team_name in @teams_changed")
    new_games_df = games_tapps_df[cols].query("team_name in @teams_changed")
    changed_games_df = new_games_df.merge(old_games_df, how="inner", on="team_name", suffixes=('_new', '_old'))
else:
    print("No previous schedule saved")

# Show changes if any...
changed_games_df

No previous schedule saved


### 5.3. Write a TeamAPP Schedule CSV & Datafarmes Pickles

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [23]:
import shutil

print('Saving TeamAPP schedule CSV file and Dataframes with id:', id_file)
for f in [file_csv, file_upcoming_pkl, file_games_tapps]:
  if os.path.exists(f):
    print("Backup file", f)
    shutil.copy(f, f + ".bak")

print('Saving CSV TeamApp schedule:', file_csv)
games_tapps_df.to_csv(file_csv, index=False)

print('Saving dataframe pickle:', file_upcoming_pkl)
upcoming_games_df.to_pickle(file_upcoming_pkl)
print('Saving dataframe pickle:', file_games_tapps)
games_tapps_df.to_pickle(file_games_tapps)

print(f"Finished saving CSV and DATA-FRAMNE files: {now.strftime('%d/%m/%Y, %H:%M:%S')}")

Saving TeamAPP schedule CSV file and Dataframes with id: 2024_04_07-06:34:52
Saving CSV TeamApp schedule: Coburg_Rep/2023-24/schedule-teamsapp-2024_04_07-06:34:52.csv
Saving dataframe pickle: Coburg_Rep/2023-24/upcoming_games_df-2024_04_07-06:34:52.pkl
Saving dataframe pickle: Coburg_Rep/2023-24/games_tapps_df-2024_04_07-06:34:52.pkl
Finished saving CSV and DATA-FRAMNE files: 07/04/2024, 06:34:52


# ------------ END FIXTURE PUBLISHING ------------

### Check a particular team

In [ ]:
team = "U10 Girls Gold"

print(games_tapps_df.query("team_name == @team")['description'].values[0])
games_tapps_df.query("team_name == @team")[['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
